## 순환신경망 구현 및 학습

### Tensorflow 설치

- TensorFlow 설치 공식 문서: [Click me!!](https://www.tensorflow.org/install/pip?hl=ko&_gl=1*1rsd120*_up*MQ..*_ga*MTY3MzQxNDUwNi4xNzIyMjM2Njgy*_ga_W0YLR4190T*MTcyMjIzNjY4Mi4xLjAuMTcyMjIzNjY4Mi4wLjAuMA..#windows-native)

- 한국 블로그: [https://chancoding.tistory.com/5](https://chancoding.tistory.com/5)

In [1]:
# !nvidia-smi

In [2]:
# TensorFlow에는 최신 버전의 pip가 필요하므로 최신 버전을 실행하려면 pip 설치를 업그레이드
# !pip install --upgrade pip

In [3]:
# For GPU users
# !conda install tensorflow[and-cuda] -y

# 또는
#!pip install  tensorflow[and-cuda] -y

# For CPU users
#!pip install tensorflow

In [4]:
# conda 가상환경인 경우
#!conda install tensorflow -y

# 가상환경이 아닌 경우
#!pip install tensorflow -y

In [5]:
# Tensorflow 삭제할 경우
# !pip uninstall tensorflow -y
#!conda remove tensorflow -y

In [6]:
import tensorflow as tf

In [7]:
# 설치된 Tensorflow 버전 확인
print(tf.__version__)

2.4.1


### Tensorflow 2.4.1 $\to$ `NotImplementError` 대처 방법

- 참고 블로그: [NotImplementedError: Cannot convert a symbolic Tensor의 해결 방법](https://riverside13.tistory.com/entry/NotImplementedError-Cannot-convert-a-symbolic-Tensor%EC%9D%98-%ED%95%B4%EA%B2%B0-%EB%B0%A9%EB%B2%95)

In [8]:
#!pip uninstall numpy -y

In [9]:
#!pip install numpy==1.19.5
#!conda install numpy=1.19.5 -c conda-forge -y

## 하이퍼 파라미터

In [10]:
EPOCHS = 10

In [11]:
'''
각 단어의 등장 횟수 측정
가장 많이 등장하는 단어 순으로 정렬
등장 횟수가 많은 순으로 1만개 단어만을 추출
(1만번째 이하 단어는 버림)
'''
NUM_WORDS = 10000 # 1만개

## 데이터셋 준비

In [12]:
# Imdb 데이터셋 로딩
imdb = tf.keras.datasets.imdb

In [13]:
# Imdb vocab dict 로딩 (단어 사전)
word_dic = tf.keras.datasets.imdb.get_word_index()

In [14]:
# 로딩된 단어 사전 확인
word_dic

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [15]:
# word_index 변환 (key, value 맞바꾸기)
dic_index_to_word = {
    value: key for key, value in word_dic.items()
}

In [16]:
# 변환된 인덱스 사전 확인
dic_index_to_word

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [17]:
# 텐서 플로우 데이터 로더 작동 확인
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/kafa46/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/kafa46/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do

In [18]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

## 데이터 로더 구현

In [19]:
# 데이터셋 생성 함수 작성
def data_loader(
    x_train, y_train, 
    x_test, y_test,
    num_words=NUM_WORDS,value=0, padding='pre', max_len=32, shuffle_num=10000, batch_size=32
):
    '''loading data & create tensorflow dataset'''
    
    # 데이터 로딩
    #(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

    # 학습 데이터셋 전처리
    x_train = tf.keras.preprocessing.sequence.pad_sequences(
        x_train, # 전처리할 데이터
        value=value,# 패딩에 사용할 값, 우리는 0 으로 채웁니다.
        padding=padding, # 패딩 방법: 빈칸이 있으면 뒷쪽에 value로 채움
        maxlen=max_len # 한 문장의 최대 길이, 우리는 32개 단어로 설정
    )
    
    # 테스트 데이터셋 전처리
    x_test = tf.keras.preprocessing.sequence.pad_sequences(
        x_test,
        value=value,
        padding=padding,
        maxlen=max_len
    )

    # 텐서 플로우 데이터셋 생성
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(shuffle_num).batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    
    return train_ds, test_ds

In [20]:
train_ds, test_ds = data_loader(x_train, y_train,x_test, y_test)

2024-07-29 17:26:32.631813: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-29 17:26:32.632078: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 17:26:32.633368: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## 데이터 유틸리티 구현

In [21]:
# 데이터 확인
print(x_train.shape, y_train.shape)
print(x_train[0], y_train[0])

(25000,) (25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1


In [22]:
# 데이터를 문장으로 복원
sentence_list = []
for index in x_train[0]:
    word = dic_index_to_word[index]
    sentence_list.append(word)

In [23]:
# 위와 같은 표현 -> List comprehension 활용
sentence_list = [dic_index_to_word[index] for index in x_train[0]]

In [24]:
# 문장을 구성하는 단어 확인
sentence_list

['the',
 'as',
 'you',
 'with',
 'out',
 'themselves',
 'powerful',
 'lets',
 'loves',
 'their',
 'becomes',
 'reaching',
 'had',
 'journalist',
 'of',
 'lot',
 'from',
 'anyone',
 'to',
 'have',
 'after',
 'out',
 'atmosphere',
 'never',
 'more',
 'room',
 'and',
 'it',
 'so',
 'heart',
 'shows',
 'to',
 'years',
 'of',
 'every',
 'never',
 'going',
 'and',
 'help',
 'moments',
 'or',
 'of',
 'every',
 'chest',
 'visual',
 'movie',
 'except',
 'her',
 'was',
 'several',
 'of',
 'enough',
 'more',
 'with',
 'is',
 'now',
 'current',
 'film',
 'as',
 'you',
 'of',
 'mine',
 'potentially',
 'unfortunately',
 'of',
 'you',
 'than',
 'him',
 'that',
 'with',
 'out',
 'themselves',
 'her',
 'get',
 'for',
 'was',
 'camp',
 'of',
 'you',
 'movie',
 'sometimes',
 'movie',
 'that',
 'with',
 'scary',
 'but',
 'and',
 'to',
 'story',
 'wonderful',
 'that',
 'in',
 'seeing',
 'in',
 'character',
 'to',
 'of',
 '70s',
 'musicians',
 'with',
 'heart',
 'had',
 'shadows',
 'they',
 'of',
 'here',
 

In [25]:
review_sentence = ' '.join(sentence_list)

In [26]:
review_sentence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have t

In [27]:
# 위 내용을 함수로 작성
def convert_indices_to_sentence(index_vocab, index_list):
    sentence_list = [index_vocab[index] for index in index_list]
    return ' '.join(sentence_list)

In [28]:
# 문장 복원 함수 테스트
convert_indices_to_sentence(dic_index_to_word, x_train[0])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have t

## 모델 정의

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(input_dim=NUM_WORDS, output_dim=16)
        
        # LSTM 레이어 활용
        # https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
        # output_dim: 출력 차원(dimensionality of the output space.)
        self.rnn = tf.keras.layers.LSTM(32) 
        #self.rnn = tf.keras.layers.SimpleRNN(32)
        #self.rnn = tf.keras.layers.GRU(32)
        
        # 활성함수 정의 (Softmax)
        # 출력 차원: 2차원 (Positive, Negative)
        # imdb 데이터셋: 영화 리뷰가 긍정/부정인지 판정 --> 출력 길이를 2로 설정
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        
        ################################################################
        # 좀더 복잡한(Deep) 신경망을 구성할 경우 성능향상 가능         #
        ################################################################
        
        return self.dense(x)

## 학습, 테스트 루프 정의

In [30]:
# 학습 루프 정의
# @tf.function --> https://www.tensorflow.org/guide/function?hl=ko
# 사용자가 만든 함수를 tensorflow가 최적화 해주는 데코레이터로 생각하면 쉽다.

# 학습 루프 정의
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    
    # 미분을 위한 GradientTape을 적용
    # 컨텍스트(context) 안에서 실행된 모든 연산을 테이프(tape)에 기록
    with tf.GradientTape() as tape:
    
        # 1. 예측 (prediction)
        predictions = model(inputs, training=True)
        
        # 2. Loss 계산
        loss = loss_object(labels, predictions)
    
    # 3. 그라디언트(gradients) 계산
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 4. 오차역전파(Backpropagation) - weight 업데이트
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # loss와 accuracy를 업데이트
    train_loss(loss)
    train_accuracy(labels, predictions)

    
# 테스트 루프 정의
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    
    # 1. 예측 (prediction)
    # Test셋에 대해서는 gradient를 계산 및 backpropagation 하지 않음
    predictions = model(images, training=False)
    
    # loss와 accuracy를 업데이트
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [31]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [32]:
best_loss = np.inf
for epoch in range(EPOCHS):
    
    train_ds, test_ds = data_loader(x_train, y_train,x_test, y_test)
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100)
    
    )
    
    # Check best model
    if test_loss.result() < best_loss:
        best_loss = test_loss.result()
        best_model = model
        print(f'Best model saved at Test Loss: {test_loss.result()}')
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

2024-07-29 17:26:33.624481: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-07-29 17:26:33.647900: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699850000 Hz


Epoch 1, Loss: 0.49988386034965515, Accuracy: 74.39600372314453, Test Loss: 0.4332514703273773, Test Accuracy: 79.41600036621094
Best model saved at Test Loss: 0.4332514703273773
Epoch 2, Loss: 0.36447271704673767, Accuracy: 83.80400085449219, Test Loss: 0.44166329503059387, Test Accuracy: 79.0
Epoch 3, Loss: 0.29812413454055786, Accuracy: 87.52400207519531, Test Loss: 0.4780960977077484, Test Accuracy: 78.33599853515625
Epoch 4, Loss: 0.242326021194458, Accuracy: 90.26000213623047, Test Loss: 0.5385760068893433, Test Accuracy: 77.64800262451172
Epoch 5, Loss: 0.18847693502902985, Accuracy: 92.86800384521484, Test Loss: 0.6662089824676514, Test Accuracy: 76.38400268554688
Epoch 6, Loss: 0.14400821924209595, Accuracy: 94.5999984741211, Test Loss: 0.7893995046615601, Test Accuracy: 76.2239990234375
Epoch 7, Loss: 0.1032751128077507, Accuracy: 96.20000457763672, Test Loss: 0.8980758190155029, Test Accuracy: 75.3239974975586
Epoch 8, Loss: 0.0742078647017479, Accuracy: 97.29199981689453, T

## 학습이 종료되었으니 이제는 모델을 가지고 놀기^^

In [33]:
model = best_model

In [34]:
# test dataset에서 첫번째 미니배치를 가져옴
x_test, y_test = next(iter(test_ds))

In [35]:
# 텐서를 넘파이로 변환
x_test_temp = x_test.numpy()

In [36]:
# 텐서를 넘파이로 변환
y_test_temp = y_test.numpy()

In [37]:
# Batch
print(x_test_temp.shape)
print(x_test_temp)
print('자료형: {}'. format(type(x_test_temp)))

(32, 32)
[[1668   14   31 ...   14    6  717]
 [1732   22   49 ...  125    4 3077]
 [ 704    7  101 ...    9   57  975]
 ...
 [ 660    4  403 ...   85   74 2129]
 [ 812   27  205 ...  195  790  158]
 [   8  901   53 ...  119   14   20]]
자료형: <class 'numpy.ndarray'>


In [38]:
x_test_temp[0]

array([1668,   14,   31,   23,   27, 7479,   29,  220,  468,    8,  124,
         14,  286,  170,    8,  157,   46,    5,   27,  239,   16,  179,
          2,   38,   32,   25, 7944,  451,  202,   14,    6,  717],
      dtype=int32)

In [39]:
# 문장 확인
convert_indices_to_sentence(dic_index_to_word, x_test_temp[0])

'carry as by are be favourites all family turn in does as three part in another some to be probably with world and her an have faint beginning own as is sequence'

In [40]:
# 정답 확인
y_test_temp[0]

0

In [41]:
# 예측해 보기
inference = model(x_test_temp, training=False)
inference

<tf.Tensor: shape=(32, 2), dtype=float32, numpy=
array([[9.16236043e-02, 9.08376396e-01],
       [7.58687513e-09, 1.00000000e+00],
       [5.35331853e-02, 9.46466804e-01],
       [2.60099828e-01, 7.39900172e-01],
       [1.57861724e-10, 1.00000000e+00],
       [9.98099029e-01, 1.90103066e-03],
       [9.98624206e-01, 1.37575797e-03],
       [9.99891996e-01, 1.07985594e-04],
       [6.69884175e-05, 9.99933004e-01],
       [1.68365514e-05, 9.99983191e-01],
       [4.42359169e-06, 9.99995589e-01],
       [7.83452094e-01, 2.16547936e-01],
       [9.99986529e-01, 1.34399934e-05],
       [9.99994993e-01, 4.95423183e-06],
       [1.09202312e-02, 9.89079773e-01],
       [9.99981880e-01, 1.80902680e-05],
       [2.84688394e-05, 9.99971509e-01],
       [9.99970913e-01, 2.91292417e-05],
       [9.99981761e-01, 1.82264812e-05],
       [9.99490023e-01, 5.09982754e-04],
       [5.28147496e-07, 9.99999523e-01],
       [6.47749112e-04, 9.99352276e-01],
       [1.54277086e-01, 8.45722914e-01],
       [

In [42]:
# softmax로 예측한 inference로부터 판단값(argmax) 추출
results = tf.argmax(inference, 1)
results

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1])>

In [43]:
# 정답
y_test_temp

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [44]:
result_numpy = results.numpy()

In [45]:
results_zipped = list(zip(result_numpy, y_test_temp))

In [46]:
for x, y in results_zipped:
    print(f'예측값: {x}, 정답: {y}')

예측값: 1, 정답: 0
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 1, 정답: 0
예측값: 1, 정답: 1
예측값: 0, 정답: 1
예측값: 0, 정답: 1
예측값: 0, 정답: 0
예측값: 1, 정답: 0
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 0, 정답: 0
예측값: 0, 정답: 0
예측값: 0, 정답: 0
예측값: 1, 정답: 1
예측값: 0, 정답: 0
예측값: 1, 정답: 1
예측값: 0, 정답: 0
예측값: 0, 정답: 0
예측값: 0, 정답: 0
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 0, 정답: 0
예측값: 1, 정답: 1
예측값: 1, 정답: 1
예측값: 0, 정답: 0
예측값: 1, 정답: 1
예측값: 1, 정답: 1


In [47]:
x_test_temp

array([[1668,   14,   31, ...,   14,    6,  717],
       [1732,   22,   49, ...,  125,    4, 3077],
       [ 704,    7,  101, ...,    9,   57,  975],
       ...,
       [ 660,    4,  403, ...,   85,   74, 2129],
       [ 812,   27,  205, ...,  195,  790,  158],
       [   8,  901,   53, ...,  119,   14,   20]], dtype=int32)

In [48]:
for index in range(len(x_test_temp)):
    review = convert_indices_to_sentence(dic_index_to_word, x_test_temp[index])
    x, y = results_zipped[index]
    print('==' * 50)
    print(f'리뷰: {review}, \n 예측값: {x}, 정답: {y}')

리뷰: carry as by are be favourites all family turn in does as three part in another some to be probably with world and her an have faint beginning own as is sequence, 
 예측값: 1, 정답: 0
리뷰: suit you good 2 which why super as it main of my i i  if time screenplay in same this remember assured have action one in realistic that better of lessons, 
 예측값: 1, 정답: 1
리뷰: class br think cop for games make southern things to it jolly who and if is boyfriend you which is tony by this make residents too not make above it even background, 
 예측값: 1, 정답: 1
리뷰: states by well potential can when it never means movie so night bad he and daughter film of unusual are of goes her them such of number big bad one left bloody, 
 예측값: 1, 정답: 0
리뷰: only deliver sleazy has thrown not wonder classic as b satisfied at main that i i their among among without didn't later if for very and didn't clearly and didn't forget didn't, 
 예측값: 1, 정답: 1
리뷰: real that real they there's at maybe those are of journey as on thing me